# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
# Your code here

# implement Norvig
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

class Norvig:
    def __init__(self, filename):
        self.WORDS = Counter(words(open(filename).read()))
        self.N = sum(self.WORDS.values())
    
    def P(self, word): 
        "Probability of `word`."
        return self.WORDS[word] / self.N
    
    def correction(self, word): 
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=self.P)
    
    def candidates(self, word): 
        "Generate possible spelling corrections for word."
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or set([word]))
    
    def known(self, words): 
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in self.WORDS)
    
    def edits1(self, word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)
    
    def edits2(self, word): 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

norvig = Norvig('big.txt') #big.txt from norvig

In [2]:
# implement bigram model:
import pandas as pd
from tqdm.notebook import tqdm

class Bigram:
    def __init__(self, filename):
        self.DICT = {}
        self.N = 0
        df = pd.read_csv(filename, sep='\t', header=None, encoding='mbcs')
        for _, row in tqdm(df.iterrows(), total = len(df)):
            self.N += row[0]
            self.DICT[(row[1], row[2])] = row[0]
    
    def P(self, word1, word2): 
        return self.DICT.get((word1, word2), 0) / self.N

    def candidates1(self, word1):
        return set(w2 for w1, w2 in self.DICT.keys() if w1 == word1)

    def candidates2(self, word2):
        return set(w1 for w1, w2 in self.DICT.keys() if w2 == word2)

bigram = Bigram('bigrams.txt') #bigrams.txt from moodle

  0%|          | 0/1020385 [00:00<?, ?it/s]

In [3]:
import numpy as np

def norm(x):
    if x.sum() == 0:
        return x
    return x / x.sum()

def correct(str, norvig=norvig, bigram=bigram): # use two models together
    w = words(str)
    n = len(w)
    
    corrections = [list(norvig.candidates(i)) for i in w] # possible corrections for every word according to norvig
    norvig_prob = [np.array(list(map(norvig.P, i))) for i in corrections] # probabilities for corrections for every word according to norvig
    norvig_prob = list(map(norm, norvig_prob)) # normalized

    bigram_prob = [np.zeros(i.size) for i in norvig_prob] # set up probability array for bigram
    bpv = np.vectorize(bigram.P) # make bigram.P numpy-compatible
    
    for i in range(n-1): # for every pair of words
        a = np.array(corrections[i]) # for all their possible corrections
        b = np.array(corrections[i+1])
        mat = bpv(a[:, np.newaxis], b) # build a matrix of cross-probabilities
        bigram_prob[i] += mat.sum(axis=1) # sum it by rows and columns and add to total bigram probabilities
        bigram_prob[i+1] = mat.sum(axis=0) 

    bigram_prob = list(map(norm, bigram_prob)) # normalized

    total_prob = [norvig_prob[i] * bigram_prob[i] for i in range(n)] # multiply probabilities 
    total_prob = list(map(norm, total_prob)) # normalized

    result = []
    for i in range(n):
        if len(corrections[i]) == 0:
            result.append(w[i])
        else:
            idx = total_prob[i].argmax()
            result.append(corrections[i][idx])

    if 0: #debug
        print(corrections)
        print(norvig_prob)
        print(bigram_prob)
            
    return ' '.join(result)

correct("sray safe anf pay taces") #check if it actually works

'stay safe and pay taxes'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

My approach is to use Norvig's solution to pefrform baseline error correction, then apply N-gram model to select the most context appropriate correction.

Norvig's solution on itself is a decent spelling correction algorithm, capable of fixing most spelling errors. The only problem which it has is lack of context-sensitivity - a strong side of N-gram models. By combining this two approaches, we could create a good context-sensitive spelling correction model.

My implementation works by first building a list of possible corrections for each word using Norvig, then applying Bi-grams approach to calculate probabilities for each word pair, keeping running totals for each correction for each word. After that, it selects the most probable word for each position, buy multiplying Norvig scores by Bi-grams scores and picking the likeliest word.

This is a simple and straightforward solution to combine Norvig approach with N-grams method.

Bi-grams should be enough to guess the right correction for a word, since spelling correction is used most of the time on short phrases (like when googling), where the adjacent words is enough to infer the meaning of a word. Implementing tri-grams or more-grams could potentially improve the accuracy on larger sentences, but would make algorithm less usable on smaller sets and hinder performace severely.

Possible improvemens: 
1. Use a better dataset (preferably the same one for Norvig part and Bigram part) # autocorrection datasets are hard to find
2. Modify the norvig to produce weighted scores (based on if they are direct, 1-edit or 2-edit) # I tried, but it resulted in worse performance

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [4]:
# Your code here
def norvig_wrapper(str, norvig=norvig): #baseline
    return ' '.join(map(norvig.correction, words(str)))

norvig_wrapper("sray safe anf pay taces")

'say safe and pay faces'

In [5]:
import random

def noize(text, p = 0.15):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    str = ''
    for l in text:
        if l in letters:
            if random.random() < p:
                str += random.choice(letters)
            else:
                str += l
        else:
            str += l
    return str

In [6]:
# totally not stolen from random place in the internet
walloftext = "anarchism is political philosophy and movement that is sceptical of authority and rejects all involuntary coercive forms of hierarchy anarchism calls for the abolition of the state which it holds to be undesirable unnecessary and harmful it is usually described alongside libertarian marxism as the libertarian wing libertarian socialism of the socialist movement and as having historical association with anti capitalism and socialism the history of anarchism goes back to prehistory when humans arguably lived in anarchistic societies long before the establishment of formal states realms or empires with the rise of organised hierarchical bodies scepticism toward authority also rose but it was not until the th century that self conscious political movement emerged during the latter half of the th and the first decades of the th century the anarchist movement flourished in most parts of the world and had significant role in workers struggles for emancipation various anarchist schools of thought formed during this period anarchists have taken part in several revolutions most notably in the spanish civil war whose end marked the end of the classical era of anarchism in the last decades of the th and into the st century the anarchist movement has been resurgent once more anarchism employs diversity of tactics in order to meet its ideal ends which can be broadly separated into revolutionary and evolutionary tactics there is significant overlap between the two which are merely descriptive revolutionary tactics aim to bring down authority and state having taken violent turn in the past evolutionary tactics aim to prefigure what an anarchist society would be like anarchist thought criticism and praxis have played part in diverse areas of human society criticisms of anarchism include claims that it is internally inconsistent violent or utopian etymology terminology and definition wilhelm weitling an example of writer who added to anarchist theory without using the exact term the etymological origin of anarchism is from the ancient greek anarkhia meaning without ruler composed of the prefix an without and the word arkhos leader or ruler the suffix ism denotes the ideological current that favours anarchy anarchism appears in english from as anarchisme and anarchy from early english usages emphasised sense of disorder various factions within the french revolution labelled their opponents as anarchists although few such accused shared many views with later anarchists many revolutionaries of the th century such as william godwin and wilhelm weitling would contribute to the anarchist doctrines of the next generation but they did not use anarchist or anarchism in describing themselves or their beliefs the first political philosopher to call himself an anarchist was pierre joseph proudhon marking the formal birth of anarchism in the mid th century since the and beginning in france libertarianism has often been used as synonym for anarchism and its use as synonym"
target = np.array(words(walloftext))

random.seed(42)
wallofnoize = noize(walloftext, p = 0.15)

In [7]:
out = np.array(words(wallofnoize))
score = np.mean(out == target)
print(f"Do nothing: {score}")

Do nothing: 0.48283261802575106


In [8]:
out = np.array(words(norvig_wrapper(wallofnoize)))
score = np.mean(out == target)
print(f"Norvig: {score}")

Norvig: 0.7918454935622318


In [9]:
out = np.array(words(correct(wallofnoize)))
score = np.mean(out == target)
print(f"Bigram + Norvig: {score}")

Bigram + Norvig: 0.7918454935622318


The result is somewhat underwhelming, but it is present. This could probably be fixed with a better dataset(s) but i couldn't find anything that is as usable as what is provided.